> **DO YOU USE GITHUB?**  
If True: print('Remember to make your edits in a personal copy of this notebook')  
Else: print('You don't have to understand. Continue your life.')

# Module 6: Web Scraping 1

In this module you will be introduced to `web scraping`: 
- What it web scraping?
- How to web scrape?
- Why is web scrpaing important to master as a data scientist?

Readings for `session 6+7+8`:
- [Python for Data Analysis, chapter 6](https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf)
- [A Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
- [An introduction to web scraping with Python](https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5)
- [Introduction to Web Scraping using Selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

Video materiale from `ISDS 2020`:
- [Web Scraping 1](https://bit.ly/ISDS2021_6)
- [Web Scraping 2](https://bit.ly/ISDS2021_7)
- [Web Scraping 3](https://bit.ly/ISDS2021_8)

Other ressources:
- [Nicklas Webpage](https://nicklasjohansen.netlify.app/)
- [Data Driven Organizational Analysis, Fall 2021](https://efteruddannelse.kurser.ku.dk/course/2021-2022/ASTK18379U)
- [Master of Science (MSc) in Social Data Science](https://www.socialdatascience.dk/education)

## Ethical Considerations
* If a regular user can’t access it, we shouldn’t try to get it [That is considered hacking](https://www.dr.dk/nyheder/penge/gjorde-opmaerksom-paa-cpr-hul-nu-bliver-han-politianmeldt-hacking). 
* Don't hit it to fast: Essentially a DENIAL OF SERVICE attack (DOS). [Again considered hacking](https://www.dr.dk/nyheder/indland/folketingets-hjemmeside-ramt-af-hacker-angreb). 
* Add headers stating your name and email with your requests to ensure transparency. 
* Be careful with copyrighted material.
* Fair use (take only the stuff you need)
* If monetizing on the data, be careful not to be in direct competition with whom you are taking the data from.

<img src="https://github.com/snorreralund/images/raw/master/Sk%C3%A6rmbillede%202017-08-03%2014.46.32.png"/>

## The Web Scraping Recipe

To scrape information from the web is:
1. **MAPPING**: Finding URLs of the pages containing the information you want.
2. **DOWNLOAD**: Fetching the pages via HTTP.
3. **PARSE**: Extracting the information from HTML.  
  
  
You could also add `connection`, `storing`, `logging`, etc.        
   


### Packages used
Today we will mainly build on the python skills you have gotten so far, and tomorrow we will look into more specialized packages.

* for connecting to the internet we use: **requests**
* for parsing: **beautifulsoup** and **regex**
* for automatic browsing / screen scraping: **selenium** 
* for mitigating errors we use: **time**

We will write our scrapers with basic python, for larger projects consider looking into the packages **scrapy**

In [10]:
# check that you can import these lbraries
# otherwise you they can easily be installed using pip
# example: https://pypi.org/project/beautifulsoup4/

import requests
from bs4 import BeautifulSoup
import re
import selenium
import time
import pandas as pd

## Connecting to the Internet


**Connecting to the internet** **HTTP**

*URL* : the adressline in our browser.

Via HTTP we send a **get** request to an *address* with *instructions* ( - or rather our dns service provider redirects our request to the right address)
*Address / Domain*: www.google.com

*Instructions*: /search?q=who+is+mister+miyagi

*Header*: information send along with the request, including user agent (operating system, browser), cookies, and prefered encoding.

*HTML*: HyperTextMarkupLanguage the language of displaying web content. More on this tomorrow.


In [11]:
import requests
response = requests.get('https://www.google.com')
#response.text

In [12]:
import requests
response = requests.get('https://isdsucph.github.io/isds2021/')
#response.text

##  Static Webpage Example

Visit the following website (https://www.basketball-reference.com/leagues/NBA_2018.html).

The page displays tables of data that we want to collect.
Tomorrow you will see how to parse such a table, but for now I want to show you a neat function that has already implemented this.

In [13]:
url = 'https://www.basketball-reference.com/leagues/NBA_2018.html' # link to the website
dfs = pd.read_html(url) # parses all tables found on the page.
dfs[0]

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Toronto Raptors*,59,23,0.720,—,111.7,103.9,7.29
1,Boston Celtics*,55,27,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers*,52,30,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers*,50,32,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers*,48,34,0.585,11.0,105.6,104.2,1.18
5,Miami Heat*,44,38,0.537,15.0,103.4,102.9,0.15
6,Milwaukee Bucks*,44,38,0.537,15.0,106.5,106.8,-0.45
7,Washington Wizards*,43,39,0.524,16.0,106.6,106.0,0.53
8,Detroit Pistons,39,43,0.476,20.0,103.8,103.9,-0.26
9,Charlotte Hornets,36,46,0.439,23.0,108.2,108.0,0.07


If we did not have a neat function we would have to navigate the website to point at the data we wanted to collect. Below I show how to find the headline of the table. This is something you will learn about in session 7.

In [14]:
url = 'https://www.basketball-reference.com/leagues/NBA_2018.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('h2')[0].text

'Conference Standings'

## Navigating websites to collect links
Now I will show you a few common ways of finding the links to the pages you want to scrape.

### Building URLS using a recognizable pattern.
A nice trick is to understand how urls are constructed to communicate with a server. 

Lets look at how [jobindex.dk](https://www.jobindex.dk/) does it. We simply click around and take note at how the addressline changes.

This will allow us to navigate the page, without having to parse information from the html or click any buttons.

* / is like folders on your computer.
* ? entails the start of a query with parameters 
* = defines a variable: e.g. page=1000 or offset = 100 or showNumber=20
* & separates different parameters.
* \+ is html for whitespace

In [15]:
# Mapping exercise
url = 'https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=2&q=python'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
soup


<!DOCTYPE html>

<html lang="da-DK">
<head>
<title>Ledige job - Python - Storkøbenhavn, side 2 ud af 7 | Jobindex</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<link href="/res/bootstrap-multiselect/dist/css/bootstrap-multiselect.css?h=6a6b68a249811e054fa8d759452816a9248c0748" rel="stylesheet"/><link href="/res/select2/dist/css/select2.min.css?h=a170ecdd58f00519741ed4b63abc064ef35db1a9" rel="stylesheet"/><link href="/res/bootstrap-datepicker/dist/css/bootstrap-datepicker3.standalone.min.css?h=f02cbfe4614ade97b3e5823be92702ae7bd445cd" rel="stylesheet"/><link href="/res/mapbox-gl/dist/mapbox-gl.css?h=0221a0dab467f93c80e8f5264c4f146e6d11496d" rel="stylesheet"/><link href="/res/font-awesome/css/font-awesome.css?h=ee906a8196d0fbd581c27a9d5615db4c250860f2" rel="stylesheet"/><link href="/css/_scss/fonts/roboto.css?h=e5bcd6527330b9ea940dd0de1cc29edbdd15519b" rel="stylesheet"/><link href="/css/_scss/fonts/frank_ruhl_libre.css?h=f908f8924442cd3fc8a7a73091

In [16]:
jobs = int(soup.find('span',attrs={'class':'d-md-none'}).text[0:3])
jobs

121

In [17]:
# 20 jobs per page
for i in range(round(jobs/20)+1):
    print(i)

0
1
2
3
4
5
6


In [18]:
for i in range(round(jobs/20)+1):
    print('https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=' + str(i) +'&q=python')

https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=0&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=1&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=2&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=3&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=4&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=5&q=python
https://www.jobindex.dk/jobsoegning/storkoebenhavn?page=6&q=python


## Good practices
* Transparency: send your email and name in the header so webmasters will know you are not a malicious actor.
* Ratelimiting: Make sure you don't hit their servers to hard.
* Reliability: 
    * Make sure the scraper can handle exceptions (e.g. bad connection) without crashing.
    * Keep a log.
    * Store raw data.


## Logging
Even if logging is not important for the below exercises, get in the habit of using this class for connecting to the internet, to practice logging your activity.

You should run `pip install scraping_class` to install the module to be used.

In [19]:
import scraping_class
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)

# Exercise Set 6: Web Scraping 1

In this Exercise Set we shall practice our webscraping skills utiilizing only basic python.  
We shall cover variations between static and dynamic pages and build. 

## Exercise Section 6.1: Scraping Jobnet.dk

This exercise you get to practice locating the request that the JavaScript sends to get the job data that it builds the joblistings from. You should use the **>Network Monitor<** tool in your browser. I recomend using Chrome.

Furthermore you practice spotting how the pagination is done, without clicking on the next page button, but instead changing a small parameter in the URL.

> **Ex. 6.1.1:** Go to  www.jobnet.dk and investigate the page. Start your `mapping`. Figure our what url you need to scrape to collect jobposting data. Sometimes this can be hard and requires you to inspect the page.

> **Ex. 6.1.2.:** Use the `request` module to collect the first 20 results and unpack the relevant `json` data into a `pandas` DataFrame.

> **Ex. 6.1.3.:** How many results do you find in total? Store this number as 'n_listings' for later use.

In [32]:
# 6.1.1
import pprint

url = "https://job.jobnet.dk/CV/FindWork/Search?Offset=0&SortValue=BestMatch"
response = connector.get(url, "jobnet-mapping")[0].json()

for key in response.keys():
    print(key)

jobs = pd.DataFrame(response["JobPositionPostings"])
jobs




Expression
Facets
JobPositionPostings
TotalResultCount


,AutomatchType,Abroad,Weight,Title,JobHeadline,Presentation,HiringOrgName,WorkPlaceAddress,WorkPlacePostalCode,WorkPlaceCity,...,HiringOrgCVR,UserLoggedIn,AnonymousEmployer,ShareUrl,DetailsUrl,JobLogUrl,HasLocationValues,ID,Latitude,Longitude
0,0,False,1.0,Miljømedarbejder til Ishøj og Vallensbæk,Miljømedarbejder til Ishøj og Vallensbæk,<p>Ishøj og Vallensbæk Kommuner søger en erfar...,Ishøj Kommune,Baldersbækvej 6,2635,Ishøj,...,11931316,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401544,https://job.jobnet.dk/CV/FindWork/Details/5401544,True,5401544,55.6182,12.3250
1,0,False,1.0,Tilkalde vikarer søges til Opholdsstedet St. T...,Tilkalde vikarer søges til Opholdsstedet St. T...,<p>Opholdsstedet St. Torøje er et socialpædago...,OPHOLDSTEDET PÅ LINDERSVOLD,Lindersvoldvej 5,4640,Faxe,...,20462183,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5334243,https://job.jobnet.dk/CV/FindWork/Details/5334243,True,5334243,55.1844,12.0799
2,0,False,1.0,Myndighedssagsbehandlere til Borgercenter Børn...,Myndighedssagsbehandlere til Borgercenter Børn...,,Københavns Kommune,Borups Allé 177,2400,København NV,...,64942212,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401519,https://job.jobnet.dk/CV/FindWork/Details/5401519,True,5401519,55.6960,12.5253
3,0,False,1.0,Svane Pleje Silkeborg søger social- og sundhed...,Svane Pleje Silkeborg søger social- og sundhed...,,"Svane Pleje, Syd ApS",Glarmestervej 16,8600,Silkeborg,...,35837876,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401518,https://job.jobnet.dk/CV/FindWork/Details/5401518,True,5401518,56.1715,9.4900
4,0,False,1.0,Havemedarbejder søges 6-8 timer ugentligt,Havemedarbejder søges 6-8 timer ugentligt,<p>Vi søger en stabil medarbejder til at varet...,Einar Risør Finerhandel a/s,Niels Bohrs vej 1,7100,Vejle,...,30369718,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401534,https://job.jobnet.dk/CV/FindWork/Details/5401534,True,5401534,55.7220,9.5719
5,0,False,1.0,Teknisk servicemedarbejder søges til Beder-Mal...,Teknisk servicemedarbejder søges til Beder-Mal...,Om jobbet:,Aarhus Kommune,Lundshøjgårdsvej 19,8340,Malling,...,55133018,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401535,https://job.jobnet.dk/CV/FindWork/Details/5401535,True,5401535,56.0395,10.1990
6,0,False,1.0,Socialpædagoger/SSA til to barselsvikariater,Socialpædagoger/SSA til to barselsvikariater,<strong>Fynbohus søger socialpædagoger/SSA til...,"Social, Sundhed & Omsorg","Nørregade 3, Vammen",8830,Tjele,...,29189846,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401517,https://job.jobnet.dk/CV/FindWork/Details/5401517,True,5401517,56.5288,9.5739
7,0,False,1.0,Pædagoger til Lundely Børnehus - vuggestue,Pædagoger til Lundely Børnehus - vuggestue,<em>Der kommer børn til byen - vi søger efter ...,Fredensborg Kommune,Lystholm 8,3480,Fredensborg,...,29188335,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401512,https://job.jobnet.dk/CV/FindWork/Details/5401512,True,5401512,55.9697,12.4204
8,0,False,1.0,Mariendal søger pædagog til børnehave/naturbør...,Mariendal søger pædagog til børnehave/naturbør...,Byens fordele og naturens frihed<br>Mariendal<...,Frederiksberg Kommune,Dronning Olgas Vej 6,2000,Frederiksberg,...,11259979,False,False,https://job.jobnet.dk/CV/FindWork/DetailsSocia...,https://job.jobnet.dk/CV/FindWork/Details/5401504,https://job.jobnet.dk/CV/FindWork/Details/5401504,True,5401504,55.6879,12.5392
9,0,False,1.0,Studentermedhjælper,Studentermedhjælper,Job i dit sabbatår?<br><br>Har du lyst til et ...,Fredensborg Kommune,Egevangen 3B,2980,Kokkedal,...,29188335,

In [30]:
# 6.1.3



n_listings = response["TotalResultCount"]
print(n_listings)

22003


> **Ex. 6.1.4:** This exercise is about paging the results. We need to understand the websites pagination scheme. 

> Now scroll down the webpage and press the next page button. See how the parameters of the url changes as you turn the pages.

> **Ex. 6.1.5:** Design a`for` loop using the `range` function that changes this paging parameter in the URL. Use 'n_listings' from before to define the limits of the range function. Store these urls in a container. 

In [15]:
# 6.1.4 & 6.1.5

url_links = []

for page in range(round(n_listings/20)+1):
    url = f"https://job.jobnet.dk/CV/FindWork?Offset={page*20}&SortValue=BestMatch"
    url_links.append(url)

print(len(url_links))


1065


> **Ex.6.1.6:** Pick 20 random links using the `random.sample()` function and scrape their content. Use the `time.sleep()` function to limit the rate of your calls. Load all the results into a DataFrame. ***extra***: monitor the time left to completing the loop by using `tqdm.tqdm()` function.


In [23]:
# 6.1.6

import random
import tqdm
from pandas.io.json import json_normalize

random_sample = random.sample(url_links, k=20)
data = []


for p in tqdm.tqdm(random_sample):
    p1, p2 = p.split("?")
    p = "/Search?".join([p1, p2])
    response = requests.get(p).json()
    data.append(pd.json_normalize(response)) 
    time.sleep(0)
    



100%|██████████| 20/20 [00:10<00:00,  1.94it/s]


In [26]:
random_df = pd.concat(data)
random_df.head()


,JobPositionPostings,TotalResultCount,Expression.ShowNearbyCriteria,Expression.Offset,Expression.Count,Expression.SearchString,Expression.SortValue,Expression.Id,Expression.EarliestPublicationDate,Expression.HotJob,...,Facets.Municipality,Facets.PostalCode,Facets.OccupationAreas,Facets.OccupationGroups,Facets.Occupations,Facets.EmploymentType,Facets.WorkHours,Facets.WorkHourPartTime,Facets.JobAnnouncementType,Facets.WorkPlaceNotStatic
0,"[{'AutomatchType': 0, 'Abroad': False, 'Weight...",21574,True,2860,20,None,BestMatch,[],0001-01-01T00:00:00,False,...,"[{'Id': 0, 'Value': 'København', 'Count': 3240...","[{'Id': 0, 'Value': '2400', 'Count': 598}, {'I...","[{'Id': 0, 'Value': 'Sundhed, omsorg og person...","[{'Id': 0, 'Value': 'Pædagogisk arbejde', 'Cou...","[{'Id': 0, 'Value': 'Pædagog', 'Count': 993}, ...","[{'Id': 0, 'Value': 'Fastansættelse', 'Count':...","[{'Id': 0, 'Value': 'Fuldtid', 'Count': 16254}...","[{'Id': 0, 'Value': 'Deltid', 'Count': 2878}]","[{'Id': 0, 'Value': 'Almindelige vilkår', 'Cou...","[{'Id': 0, 'Value': 'false', 'Count': 21033}, ..."
0,"[{'AutomatchType': 0, 'Abroad': False, 'Weight...",21574,True,8480,20,None,BestMatch,[],0001-01-01T00:00:00,False,...,"[{'Id': 0, 'Value': 'København', 'Count': 3240...","[{'Id': 0, 'Value': '2400', 'Count': 598}, {'I...","[{'Id': 0, 'Value': 'Sundhed, omsorg og person...","[{'Id': 0, 'Value': 'Pædagogisk arbejde', 'Cou...","[{'Id': 0, 'Value': 'Pædagog', 'Count': 993}, ...","[{'Id': 0, 'Value': 'Fastansættelse', 'Count':...","[{'Id': 0, 'Value': 'Fuldtid', 'Count': 16254}...","[{'Id': 0, 'Value': 'Deltid', 'Count': 2878}]","[{'Id': 0, 'Value': 'Almindelige vilkår', 'Cou...","[{'Id': 0, 'Value': 'false', 'Count': 21033}, ..."
0,"[{'AutomatchType': 0, 'Abroad': False, 'Weight...",21574,True,15800,20,None,BestMatch,[],0001-01-01T00:00:00,False,...,"[{'Id': 0, 'Value': 'København', 'Count': 3240...","[{'Id': 0, 'Value': '2400', 'Count': 598}, {'I...","[{'Id': 0, 'Value': 'Sundhed, omsorg og person...","[{'Id': 0, 'Value': 'Pædagogisk arbejde', 'Cou...","[{'Id': 0, 'Value': 'Pædagog', 'Count': 993}, ...","[{'Id': 0, 'Value': 'Fastansættelse', 'Count':...","[{'Id': 0, 'Value': 'Fuldtid', 'Count': 16254}...","[{'Id': 0, 'Value': 'Deltid', 'Count': 2878}]","[{'Id': 0, 'Value': 'Almindelige vilkår', 'Cou...","[{'Id': 0, 'Value': 'false', 'Count': 21033}, ..."
0,"[{'AutomatchType': 0, 'Abroad': False, 'Weight...",21574,True,19840,20,None,BestMatch,[],0001-01-01T00:00:00,False,...,"[{'Id': 0, 'Value': 'København', 'Count': 3240...","[{'Id': 0, 'Value': '2400', 'Count': 598}, {'I...","[{'Id': 0, 'Value': 'Sundhed, omsorg og person...","[{'Id': 0, 'Value': 'Pædagogisk arbejde', 'Cou...","[{'Id': 0, 'Value': 'Pædagog', 'Count': 993}, ...","[{'Id': 0, 'Value': 'Fastansættelse', 'Count':...","[{'Id': 0, 'Value': 'Fuldtid', 'Count': 16254}...","[{'Id': 0, 'Value': 'Deltid', 'Count': 2878}]","[{'Id': 0, 'Value': 'Almindelige vilkår', 'Cou...","[{'Id': 0, 'Value': 'false', 'Count': 21033}, ..."
0,"[{'AutomatchType': 0, 'Abroad': False, 'Weight...",21574,True,17700,20,None,BestMatch,[],0001-01-01T00:00:00,False,...,"[{'Id': 0, 'Value': 'København', 'Count': 3240...","[{'Id': 0, 'Value': '2400', 'Count': 598}, {'I...","[{'Id': 0, 'Value': 'Sundhed, omsorg og person...","[{'Id': 0, 'Value': 'Pædagogisk arbejde', 'Cou...","[{'Id': 0, 'Value': 'Pædagog', 'Count': 993}, ...","[{'Id': 0, 'Value': 'Fastansættelse', 'Count':...","[{'Id': 0, 'Value': 'Fuldtid', 'Count': 16254}...","[{'Id': 0, 'Value': 'Deltid', 'Count': 2878}]","[{'Id': 0, 'Value': 'Almindelige vilkår', 'Cou...","[{'Id': 0, 'Value': 'false', 'Count': 21033}, ..."


> **Ex.6.1.7:** Snorre Ralund, a resaercher at SODAS, has build a connector class. Repeat 6.1.6 but try to to use Snorres connector to log your activity.


## Exercise Section 6.2: Scraping Trustpilot.com
Now for a slightly more elaborate, yet still simple scraping problem. Here we want to scrape trustpilot for user reviews. This data is very nice since it provides free labeled data (rating) to train a machine learning model to understand positive and negative sentiment. 

Here you will practice crawling a website collecting the links to each company review page, and finally locate another behind the scenes JavaScript request that gets the review data in a neat json format.

> **Ex. 6.2.1:** Visit the https://www.trustpilot.com/ website and locate the categories page.
From this page you find links to company listings.

> **Ex. 6.2.2:**
Get the category page using the `requests` module and extract each link to a specific category page from the HTML. This can be done using the basic python `.split()` string method. Make sure only links within the ***/categories/*** section are kept, checking each string using the ```if 'pattern' in string``` condition. 

*(Hint: The links are relative. You need to add the domain name)*


In [45]:

url = "https://www.trustpilot.com/categories"

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'lxml')
links = soup.find_all('a')
category_links = []

for link in links:
    if link.has_attr("href"):
        if "/categories/" in link["href"]:
            category_links.append(link["href"])



> **Ex. 6.2.3:** Get one of the category section links. Write a function to extract the links to the company review page from the HTML.

> **Ex. 6.2.4:** Figure out how the pagination is done, by following how the url changes when pressing the **next page**-button to obtain more company listings. Write a function that builds links to paging all the company listing results of each category. This includes parsing the number of subpages of each category and changing the correct parameter in the url.

(Hint: Find the maximum number of result pages, right before the next page button and make a loop change the page parameter of the url.)


In [80]:
url_new = str(url + category_links[0])

def review_link(cat):

    review_links = []

    url = "https://www.trustpilot.com/categories" + "/" + str(cat)

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('a')

    for link in links:
        if link.has_attr("href"):
            if "/review/" in link["href"]:
                review_links.append("https://www.trustpilot.com"  + link["href"]) 
                
    
    return review_links[:20]
    

review_link("cosmetics_makeup")


['https://www.trustpilot.com/review/fragrancebuy.ca',
 'https://www.trustpilot.com/review/aylabeauty.com',
 'https://www.trustpilot.com/review/www.venbafragrance.com',
 'https://www.trustpilot.com/review/balmstandard.com',
 'https://www.trustpilot.com/review/www.olfactoryfactoryllc.com',
 'https://www.trustpilot.com/review/coloradorealsoap.com',
 'https://www.trustpilot.com/review/beautystoredepot.com',
 'https://www.trustpilot.com/review/www.blankcanvascosmetics.com',
 'https://www.trustpilot.com/review/www.newdirectionsaromatics.com',
 'https://www.trustpilot.com/review/www.yougonatural.com',
 'https://www.trustpilot.com/review/cosmeticsandyou.com',
 'https://www.trustpilot.com/review/10daylashes.com',
 'https://www.trustpilot.com/review/naturaltransplants.com',
 'https://www.trustpilot.com/review/www.asraestheticsltd.com',
 'https://www.trustpilot.com/review/adamaminerals.com',
 'https://www.trustpilot.com/review/sea-kind.com',
 'https://www.trustpilot.com/review/decantx.com',
 'htt

> **Ex. 6.2.5:** Loop through all categories and build the paging links using the above defined function.

> **Ex. 6.2.6:** Randomly pick one of category listing links you have generated, and get the links to the companies listed using the other function defined. 

> **Ex. 6.2.7:** Visit one of these links and inspect the **>Network Monitor<** to locate the request that loads the review data. Use the requests module to retrieve this link and unpack the json results to a pandas DataFrame.
